In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

#HEADERS ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [4]:
# https://www.pycodemates.com/2022/01/Indeed-jobs-scraping-with-python-bs4-selenium-and-pandas.html

In [7]:
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
# driver = webdriver.Firefox(options=options)
# driver.get('http://google.com/')

In [20]:
def get_test(url, job_title, location):
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(3)
    driver.find_element(By.XPATH,'//*[@id="text-input-what"]').send_keys(job_title)
    time.sleep(3)
    locations = driver.find_element(By.XPATH,'//*[@id="text-input-where"]')
    if locations.get_attribute("value") == "":
        driver.find_element(By.XPATH,'//*[@id="text-input-where"]').send_keys(location)
    time.sleep(3)
    driver.find_element(By.XPATH,'/html/body/div').click()
    time.sleep(3)
    try:
        driver.find_element(By.XPATH,'//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element(By.XPATH,'//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url
    return current_url
    
current_url = get_test('https://indeed.com/','Data Scientist',"Los Angeles")

In [21]:
def scrape_job_details(url):
    
    resp = requests.get(url)
    content = BeautifulSoup(resp.content, 'lxml')
   
    jobs_list = []    
    for post in content.select('.job_seen_beacon'):
        try:
            data = {
                "job_title":post.select('.jobTitle')[0].get_text().strip(),
                "company":post.select('.companyName')[0].get_text().strip(),
                "rating":post.select('.ratingNumber')[0].get_text().strip(),
                "location":post.select('.companyLocation')[0].get_text().strip(),
                "date":post.select('.date')[0].get_text().strip(),
                "job_desc":post.select('.job-snippet')[0].get_text().strip(),
                "job_link":post.find(class_='jcs-JobTitle', href=True)['href']
            }
        except IndexError:
            continue          
        jobs_list.append(data)
    dataframe = pd.DataFrame(jobs_list)
    dataframe["job_link"] = 'https://www.indeed.com' + dataframe['job_link'].astype(str)
     
    return dataframe

lol = scrape_job_details(current_url)
# rename columns
lol["date"] = [x.replace('Posted','') for x in lol["date"]]
lol["job_title"] = [x.replace('new', '') for x in lol["job_title"]]

In [22]:
lol

,job_title,company,rating,location,date,job_desc,job_link
0,Health Program Analyst II - ACDC Epi & Data Un...,"Healthcare Staffing Professionals, Inc.",3.9,"Los Angeles, CA",Hiring ongoing,Liaise with surveillance web application team ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Scientist,Deloitte,4.0,"Los Angeles, CA 90013 (Downtown area)+2 locations",Today,A Master's in a quantitative field (engineerin...,https://www.indeed.com/rc/clk?jk=49b4424ef9d3b...
2,Sr. Data Scientist,AEG Worldwide,3.9,"Los Angeles, CA",1 day ago,Experience building statistical modeling and a...,https://www.indeed.com/rc/clk?jk=c06da7013743a...
3,Data Scientist - Analytics,Twitch,4.4,"Remote in Los Angeles, CA",30+ days ago,Develop scalable data pipelines to support ana...,https://www.indeed.com/rc/clk?jk=6c15755a1bb6e...
4,Data Scientist (Remote),Manufacturers Bank,3.6,"Remote in Los Angeles, CA 90071",30+ days ago,"In addition, the data scientist will assist in...",https://www.indeed.com/rc/clk?jk=a3a5dc77f000b...
5,Data Scientist - TikTok US - Tech Services,TikTok,3.2,"Los Angeles, CA+8 locations",30+ days ago,Understand underlying data sources and their l...,https://www.indeed.com/rc/clk?jk=8dce625b03da4...
6,Data Scientist/Senior Data Scientist,UCLA,4.2,"Remote in Los Angeles, CA",1 day ago,Partner with nontechnical stakeholders to set ...,https://www.indeed.com/rc/clk?jk=7f530373e5253...
7,Data Scientist,UCLA Health,4.0,"Los Angeles, CA 90095+1 location",6 days ago,This position will support the organization’s ...,https://www.indeed.com/rc/clk?jk=4001278d06a11...
8,Lead Threat Data Scientist,TikTok,3.2,"Los Angeles, CA+1 location",2 days ago,Build and train a team of data scientists and ...,https://www.indeed.com/rc/clk?jk=63a01c20ff30c...
9,Data Scientist - Predictive,Electronic Arts,3.8,"Los Angeles, CA",28 days ago,You will get hands on experience building tool...,https://www.indeed.com/rc/clk?jk=19ac3510766e3...


In [23]:
lol.iloc[6]["job_desc"]

'Partner with nontechnical stakeholders to set data objectives and develop detailed project execution plans.\nExperience with Python, R, or a similar statistical…'